# SPECT Ordered Subsets Expectation Maximisation Notebook
A notebook to demonstrate the setup and basic OSEM reconstruction of a 2-dimensional dummy image using SIRF's SPECT projector

 # What is OSEM?
The following is just a very brief explanation of the concepts behind OSEM.

PET reconstruction is commonly based on the *Maximum Likelihood Estimation (MLE)* principle. The *likelihood* is the probability to observe some measured data given a (known) image. MLE attempts to find the image that maximises this likelihood. This needs to be done iteratively as the system of equations is very non-linear.

A common iterative method uses *Expectation Maximisation*, which we will not explain here. The resulting algorithm is called *MLEM* (or sometimes *EMML*). However, it is rather slow. The most popular method to increase computation speed is to compute every image update based on only a subset of the data. Subsets are nearly always chosen in terms of the "views" (or azimuthal angles). The *Ordered Subsets Expectation Maximisation (OSEM)* cycles through the subsets. More on this in another notebook, but here we just show how to use the SIRF implementation of OSEM.

OSEM is (still) the most common algorithm in use in clinical PET.

This notebook is a skeleton for a simple OSEM reconstruction using the SIRF SPECT projector and OSEM reconstructor.
See the **PET_OSEM** and **SPECT_Acquisition** model notebooks for information

# Initial Setup

In [ ]:
import os
import glob
import numpy as np

import notebook_setup

from sirf.STIR import show_2D_array
import sirf.STIR as spect
from sirf.Utilities import examples_data_path
from sirf_exercises import cd_to_working_dir
data_path = examples_data_path('SPECT')

cd_to_working_dir("SPECT", "OSEM")

Redirect information, warning and error messages to log files

In [ ]:
msg_red = spect.MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')

Some useful function definitions

In [ ]:
def create_sample_image(image):
    '''fill the image with some simple geometric shapes.'''
    image.fill(0)
    # create a shape
    shape = spect.EllipticCylinder()
    shape.set_length(400)
    shape.set_radii((100, 40))
    shape.set_origin((0, 60, 10))

    # add the shape to the image
    image.add_shape(shape, scale = 1)

    # add another shape
    shape.set_radii((30, 30))
    shape.set_origin((60, -30, 10))
    shape.set_origin((60, -30, 10))
    image.add_shape(shape, scale = 1.5)

    # add another shape
    shape.set_origin((-60, -30, 10))
    image.add_shape(shape, scale = 0.75)

def make_cylindrical_FOV(image):
    """truncate to cylindrical FOV."""
    cyl_filter =spect.TruncateToCylinderProcessor()
    cyl_filter.apply(image)
    return image

def add_noise(proj_data,noise_factor = 1):
    """Add Poission noise to acquisition data."""
    proj_data_arr = proj_data.as_array() / noise_factor
    # Data should be >=0 anyway, but add abs just to be safe
    proj_data_arr = np.abs(proj_data_arr)
    noisy_proj_data_arr = np.random.poisson(proj_data_arr).astype('float32');
    noisy_proj_data = proj_data.clone()
    noisy_proj_data.fill(noisy_proj_data_arr);
    return noisy_proj_data

# Create ground truth and simulated data images

First, we create a template acquisition data object from file. I'll do this for you...

In [ ]:
templ_sino = spect.AcquisitionData(os.path.join(data_path,'template_sinogram.hs'))

We then use this template sinogram to create a simple ground truth image

In [ ]:
# create ground truth image
image = templ_sino.create_uniform_image()
create_sample_image(image)
image = image.zoom_image(zooms=(0.5, 1.0, 1.0)) #required for now because SPECT is 360 degree acquisiton

In [ ]:
# show the ground truth image
image_array = image.as_array()
show_2D_array('Phantom image', image_array[0,:,:])

Now we need to create the acquisition model by first creating an acquisition matrix (SPECTUBMatrix) object and apply this to a sirf Acquisition Model (AcquisitionModelUsingMatrix) object

In [ ]:
### Acquisition Model code here ###

Next, forward project our ground truth image and add noise to simulate noisy acquired data.

In [ ]:
print('projecting image...')
# project the image to obtain simulated acquisition data
# data from raw_data_file is used as a template
acq_model.set_up(templ_sino, image)
simulated_data = templ_sino.get_uniform_copy()
acq_model.forward(image, 0, 1, simulated_data)

# create noisy data
noisy_data = simulated_data.clone()
noisy_data_as_array = np.random.poisson(simulated_data.as_array())
noisy_data.fill(noisy_data_as_array)

# show simulated acquisition data
simulated_data_as_array = simulated_data.as_array()
show_2D_array('Forward projection', simulated_data_as_array[0, 0,:,:])
show_2D_array('Forward projection with added noise', noisy_data_as_array[0, 0,:,:])

We now create the reconstruction problem's objective function and use this to create an OSEM reconstructor object

In [ ]:
### Objective Function code here ###

num_subsets = 21 # number of subsets for OSEM reconstruction
num_subiters = 42 #number of subiterations (i.e two full iterations)

### OSEM reconstructor code here ###

In [ ]:
# create initialisation image and set up reconstructor
init_image = make_cylindrical_FOV(image.get_uniform_copy(1))

### Now set_up the reconstructor object using the initial image ###

And now for the reconstruction...

In [ ]:
### Reconstruction code here ###

How did we do?

In [ ]:
show_2D_array('Reconstructed image', np.squeeze(out.as_array()[0,:,:])) 

In [ ]:
#%% delete temporary files
wdpath = os.getcwd()
for filename in glob.glob(os.path.join(wdpath, "tmp*")):
    os.remove(filename) 

# Exercise: reconstruct with and without attenuation and resolution modelling
* Investigate the effect of resolution modelling and attenuation on the reconstructed image
* What happens if you have a different resolution model for the simulated data and the reconstruction?

Hint: help(spect.SPECTUBMatrix)